### Imports

In [1]:
! pip install pandas numpy
! pip install -U scikit-learn
! pip install imblearn
! pip install kaggle
! pip install streamlit
! pip install tqdm
! pip install streamlit_jupyter
! pip install streamlit-echarts
! pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
import json, glob
import numpy as np

## Instaloader api

In [3]:
from instaloader import Instaloader, Profile, ProfileNotExistsException, LoginRequiredException
L=Instaloader()
import time
from functools import lru_cache 
  
ERROR=''
@lru_cache(maxsize = 100) 
def get_info(username, timeout=0):
    global ERROR
    try:
        time.sleep(timeout)
        print("Scraping ...")
        profile = Profile.from_username(L.context, username=username)
        info = profile._asdict()
        return info
    except ProfileNotExistsException as e:
        print(e)
        ERROR="Profile doesn't exist"
    except LoginRequiredException as e:
        print(e)
        ERROR="Need to login to use instaloader, timeout from instaloader"   

### Feature enggineering

In [4]:
features =[
#     'full_name',
#     'username',
#      'id',
    
    'biography', #len
#  'blocked_by_viewer',
#  'restricted_by_viewer',
 # 'country_block',
#  'external_url',
#  'external_url_linkshimmed',
 'edge_followed_by', #count inside
 'fbid', #is none
#  'followed_by_viewer',
 'edge_follow',  #count inside
#  'follows_viewer',
 
 'has_ar_effects',
 'has_clips',
 'has_guides',
 'has_channel',
    
 # 'has_blocked_viewer',
 'highlight_reel_count',
 # 'has_requested_viewer',
 'is_business_account',
 # 'is_joined_recently',
#  'business_category_name',
#  'overall_category_name',
#  'category_enum',
#  'category_name',
 'is_private',
 'is_verified',
#  'edge_mutual_followed_by',
 'profile_pic_url', #if none
#  'profile_pic_url_hd',
#  'requested_by_viewer',
#  'should_show_category',
 # 'connected_fb_page', #if None or not
#  'edge_felix_video_timeline',
#  'edge_owner_to_timeline_media',
#  'edge_saved_media',
#  'edge_media_collections'
]

def process_entries(ent):
    ent=ent.copy()
    ent['biography']=len(ent['biography'])
    ent['fbid']=bool(ent['fbid'])
    ent['profile_pic_url']=bool(ent['profile_pic_url'])
    
    ent['edge_followed_by']=ent['edge_followed_by']['count']
    ent['edge_follow']=ent['edge_follow']['count']
    
#     ent.pop('full_name'), ent.pop('id'), ent.pop('username')
    return ent


## Collect spam users data
https://www.kaggle.com/datasets/rezaunderfit/instagram-fake-accounts-dataset

In [5]:
! mkdir -p dataset/fake/
! kaggle datasets download -d  rezaunderfit/instagram-fake-accounts-dataset --force --unzip -p dataset/fake/

fakedata=[json.load(open(f))['graphql']['user'] for f in glob.glob('dataset/fake/db/*.json')]

100%|██████████████████████████████████████| 3.72M/3.72M [00:01<00:00, 2.91MB/s]
100%|██████████████████████████████████████| 3.72M/3.72M [00:01<00:00, 2.11MB/s]


### Collect real users and scrape them
https://raw.githubusercontent.com/harshitkgupta/Fake-Profile-Detection-using-ML/master/data/users.csv

In [6]:
! mkdir -p dataset/real/ 
! wget -nc https://raw.githubusercontent.com/harshitkgupta/Fake-Profile-Detection-using-ML/master/data/users.csv -P dataset/real/  

File ‘dataset/real/users.csv’ already there; not retrieving.



In [ ]:
import os
from tqdm import tqdm

# realusertxt='dataset/real/real.txt'
realjson='dataset/real/real.json'
realcsv='dataset/real/users.csv'

if os.path.isfile(realjson):
    realdata = json.load(open(realjson))
else:
    realdata = []
    print('One time setup to be done, this may take a while...')
    
savedusers = [row['username'] for row in realdata]
    
# for realuser in tqdm(open(realusertxt).readlines()):
MAXSCRAPE=0
realdf = pd.read_csv(realcsv)
i=0
for realuser in tqdm(realdf["screen_name"].sample(frac=1).reset_index(drop=True)):
    print(realuser)
    if realuser not in savedusers:
        ret=get_info(realuser)
        if ret:
            realdata.append(ret)
            json.dump(realdata, open(realjson, 'w+'))
            print(f'[{i}]: Scraped and saved')
            i+=1
            if i >= MAXSCRAPE:
                break
        else:
            realdf=realdf[realdf["screen_name"] != realuser]
            realdf.to_csv(realcsv, index=False)
        
# json.dump(realdata, open(realjson, 'w+'))

In [8]:
fakedf = pd.DataFrame(map(process_entries, fakedata), columns=features)
fakedf['fake']=1
realdf = pd.DataFrame(map(process_entries, realdata), columns=features)
realdf['fake']=0
df = pd.concat([fakedf, realdf])
df=df.sample(frac=1).reset_index(drop=True)
df.head()

,biography,edge_followed_by,fbid,edge_follow,has_ar_effects,has_clips,has_guides,has_channel,highlight_reel_count,is_business_account,is_private,is_verified,profile_pic_url,fake
0,0,59,True,2126,False,False,False,False,0,False,False,False,True,1
1,24,95,True,3042,False,False,False,False,0,False,True,False,True,1
2,0,1211,True,1769,False,False,False,False,0,False,True,False,True,1
3,149,262,True,1113,False,False,False,False,0,False,False,False,True,1
4,0,99,True,5277,False,False,False,False,0,False,False,False,True,1


## Oversampling imbalanced data

In [9]:
from imblearn.over_sampling import SMOTE
X, y = SMOTE().fit_resample(df.drop(columns=['fake']), df['fake'])

## Model

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate, cross_val_score
scoring = {'acc': 'accuracy',
           'prec_macro': 'precision_macro',
           'rec_micro': 'recall_macro'}

cross_validate(RandomForestClassifier(), X, y, cv=5, scoring=scoring)

{'fit_time': array([0.18657541, 0.18432188, 0.18492699, 0.1844666 , 0.1832993 ]),
 'score_time': array([0.02390385, 0.01680923, 0.01732278, 0.01671243, 0.01687765]),
 'test_acc': array([0.96402878, 0.97111913, 0.97111913, 0.97472924, 0.9566787 ]),
 'test_prec_macro': array([0.96441336, 0.97119395, 0.97119395, 0.97476538, 0.95675182]),
 'test_rec_micro': array([0.96402878, 0.97114482, 0.97114482, 0.97471588, 0.9567042 ])}

In [11]:
import pickle

model=RandomForestClassifier()
model.fit(X, y)
pickle.dump(model, open('model', 'wb+'))

## Test on realdata

In [12]:
def predict(username):
    np.random.seed(1111)
    info = get_info(username)
    if info is None:
        return 404
    return model.predict_proba(pd.DataFrame([process_entries(info)], columns=features))[0][1]

(0.02, 0.99)

In [13]:
A=dict(zip(*(model.feature_names_in_, model.feature_importances_)))
{k:A[k] for k in sorted(A, key=A.get, reverse=True)}

{'edge_follow': 0.5155748611234072,
 'edge_followed_by': 0.2927940825277123,
 'has_clips': 0.08159158196520172,
 'biography': 0.06247100428521233,
 'highlight_reel_count': 0.03613317006028254,
 'is_business_account': 0.006703495075614869,
 'is_private': 0.004731804962569093,
 'fbid': 0.0,
 'has_ar_effects': 0.0,
 'has_guides': 0.0,
 'has_channel': 0.0,
 'is_verified': 0.0,
 'profile_pic_url': 0.0}

### Run the webapp

In [16]:
! source env/bin/activate && streamlit run infer.py


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://192.168.0.115:8501

Scraping ...
Scraping ...
Scraping ...
Scraping ...
Scraping ...
Scraping ...
^C
  Stopping...
